# Imports and Function Definitions

In [ ]:
import pandas as pd

import lexis_functions as lf

In [ ]:
def diff_month(s1, s2):
    s1year, s1month = s1.map(lambda x: x.year), s1.map(lambda x: x.month)
    s2year, s2month = s2.map(lambda x: x.year), s2.map(lambda x: x.month)
    
    return (s2year - s1year) * 12 + s2month - s1month

# Read in & Clean Data

In [ ]:
try:
    df = pd.read_csv(r"O:\LexisNexis\LN_NOID_DATA\original\LN_Output_Address_RelAsso_LN_InputLexisNexisCHSParticipants.DatasetNSv2.csv",
                    index_col='ssn_altkey')
except FileNotFoundError:
    df = pd.read_csv(r"C:\Users\jc4673\Documents\CHS_Lexis\LexisNexis\LN_NOID_DATA\original\LN_Output_Address_RelAsso_LN_InputLexisNexisCHSParticipants.DatasetNSv2.csv",
                    index_col='ssn_altkey')
my_cols = ['yrdeath'] + [col for col in df.columns if '_seen' in col]
df = df[my_cols].reset_index(drop=False)

In [ ]:
df.head()

In [ ]:
df_long = pd.wide_to_long(df, ['best_address_last_seen_', 'best_address_first_seen_'], i='ssn_altkey', j='num')
df_long = df_long.sort_index().dropna(subset=['best_address_last_seen_', 'best_address_first_seen_'])
df_long.columns = ['death', 'last_seen_date', 'first_seen_date']
df_long['death'] = pd.to_datetime(df_long.death, format='%Y')

In [ ]:
df_long = lf.convert_all_dates(df_long)

In [ ]:
#Distribution of number of addresses
df_long.reset_index(drop=False).num.value_counts()

# Length of Stay at This Address

lex_bestaddresslength*  


In [ ]:
df_long['lex_bestaddresslength'] = diff_month(df_long.first_seen_date, df_long.last_seen_date)

# Most recent address truncated by date of death if possible

lex_best_address_correct - Boolean indicating whether the last seen for the most recent address is "correct" (it doesn't extend beyond the date of death)  

lex_address_last_mod - Most recent date of death truncated by date of death if possible  

**These are housed in a different dataframe than lex_bestaddresslength\*, as it only applies to the most recent address and therefore each participant will have only one entry**

In [ ]:
df_last = df_long.groupby(level=0).last()

df_last['best_address_last_correct'] = df_last['last_seen_date'] < df_last['death']
df_last.head()

In [ ]:
# This is somewhat concerning
df_last.best_address_last_correct.value_counts()

In [ ]:
df_last['lex_address_last_mod'] = df_last['last_seen_date']

In [ ]:
df_last.loc[df_last['best_address_last_correct'] == False, 'lex_address_last_mod'] = df_last['death']

In [ ]:
df_last.head()

# Write to CSV

In [ ]:
df_long['lex_bestaddresslength'].to_frame().to_csv(
    r"C:\Users\jc4673\Documents\CHS_Lexis\LexisNexis\LN_NOID_DATA\derived\best_address_length_months.csv")

In [ ]:
df_last[['best_address_last_correct', 'lex_address_last_mod']].to_csv(
    r"C:\Users\jc4673\Documents\CHS_Lexis\LexisNexis\LN_NOID_DATA\derived\best_address_trunc_death.csv")